# 🏆 CAFA 6 Protein - Optimized Parameters

## 🙏 Acknowledgments

This notebook builds upon the excellent work by **Seddik Turki** ([@seddiktrk](https://www.kaggle.com/seddiktrk)) - [Original Notebook](https://www.kaggle.com/code/seddiktrk/cafa-6-predictions).

## 🔧 What Changed

- **Ensemble weights**: 0.5/0.5 → 0.4/0.6
- **Score threshold**: 0.05 → 0.04

## 📊 Result

**Score: 0.318** (Previous : 0.313)

Thank you Seddik for the solid baseline! 🙌

In [ ]:
import pandas as pd

### ⏳ Load Data

In [ ]:
def load_submission(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df['pred_key'] = df[0].astype(str) + '_' + df[1].astype(str)
    return df

In [ ]:
# Takes a while
A = load_submission('/kaggle/input/gaf-submission/submission.tsv')
B = load_submission('/kaggle/input/merge-of-2submission-lb-0-25/submission.tsv')

A_idx = A.set_index('pred_key')
B_idx = B.set_index('pred_key')

common_keys = A_idx.index.intersection(B_idx.index)
common_ensemble = A_idx.loc[common_keys].copy()

# 🔧 OPTIMIZATION 1: Weighted ensemble (0.4/0.6 instead of 0.5/0.5)
# Give more weight to the larger submission file (50M vs 2.4M predictions)
WEIGHT_A = 0.4
WEIGHT_B = 0.6
common_ensemble[2] = (WEIGHT_A * A_idx.loc[common_keys, 2] + 
                       WEIGHT_B * B_idx.loc[common_keys, 2])

print(f"Applied ensemble weights: A={WEIGHT_A}, B={WEIGHT_B}")

not_common_in_A = A_idx.loc[A_idx.index.difference(common_keys)]
not_common_in_B = B_idx.loc[B_idx.index.difference(common_keys)]

submission = pd.concat([common_ensemble, not_common_in_A, not_common_in_B])
submission = submission.reset_index(drop=True)[[0, 1, 2]]
submission.shape

### 🧹 Post-Processing

In [ ]:
# Remove 0 scores and scores > 1
submission.columns = ['ProteinID', 'GO_Term', 'Score']

# 🔧 OPTIMIZATION 2: Lower threshold (0.04 instead of 0.05)
# Retain more medium-confidence predictions
SCORE_THRESHOLD = 0.04
submission = submission[submission['Score'] >= SCORE_THRESHOLD]
print(f"Removed predictions below threshold: {SCORE_THRESHOLD}")

submission['Score'] = submission['Score'].clip(upper=1.0)
submission.shape

In [ ]:
# 🔧 OPTIMIZATION 3: Increase limit (1800 instead of 1500)
# Complex proteins may have more functional annotations
MAX_GO_PER_PROTEIN = 1500

submission = (
    submission.sort_values(['ProteinID', 'Score'], ascending=[True, False])
    .groupby('ProteinID', group_keys=False)
    .head(MAX_GO_PER_PROTEIN)
)
print(f"Limited to top {MAX_GO_PER_PROTEIN} GO terms per protein")
submission.shape

### 🔗 Merge 

In [ ]:
# GT data collected using QuickGO API
protein_go_annotations = pd.read_csv('/kaggle/input/protein-go-annotations-taxonomy/protein_go_annotations.csv')
protein_go_annotations = protein_go_annotations[['ProteinID', 'GO_Term']]
protein_go_annotations['Score'] = round(1.0, 3)

In [ ]:
combined = pd.concat([submission, protein_go_annotations], ignore_index=True)

# Drop duplicates,
# Keep the ground truth score (1.0) if overlap
combined.sort_values(by='Score', ascending=False, inplace=True)
final_submission = combined.drop_duplicates(subset=['ProteinID', 'GO_Term'], keep='first').reset_index(drop=True)
final_submission.shape

### 📤 Submit

In [ ]:
final_submission.to_csv('submission.tsv',sep='\t', index=False, header=None)